### 二维互相关运算

In [26]:
import torch
from torch import nn

def corr2d(X,K):
    h,w = K.shape
    Y = torch.zeros((X.shape[0] -h +1,X.shape[1] - w +1))
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            Y[i,j] = (X[i:i+h,j:j+w]*K).sum()
    return Y

In [8]:
X = torch.tensor([[0,1,2],[3,4,5],[6,7,8]])
K = torch.tensor([[0,1],[2,3]])
corr2d(X,K)

tensor([[19., 25.],
        [37., 43.]])

### 二维卷积层
#### 二维卷积层将输入和卷积核做互相关运算，并加上一个标量偏差来得到输出。卷积层的模型参数包括了卷积核和标量偏差。在训练模型的时候，通常我们先对卷积核随机初始化，然后不断迭代卷积核和偏差
#### 下面基于corr2d函数来实现一个自定义的二维卷积层，在构造函数__init__里我们声明weight和bias这两个模型参数。前向计算函数forward则是直接调用corr2d函数再加上偏差

In [27]:
class Conv2D(nn.Module):
    def __init__(self,kernel_size):
        super(Conv2D,self).__init__()
        self.weight = nn.Parameter(torch.randn(kernel_size))
        self.bias = nn.Parameter(torch.randn(1))
        
    def forward(self,x):
        return corr2d(x,self.weight)+self.bias

### 图像中物体边缘检测
#### 下面我们来看一个卷积层的简单应用：检测图像中物体的边缘，即找到像素变化的位置。首先我们构造一张6*8的图像。它中间4列为黑（0）,其余为白（1）

In [17]:
X= torch.ones(6,8)
X[:,2:6] = 0
print(X)

tensor([[1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.]])


#### 然后我们构造一个高和宽分别为1和2的卷积核K 当它与输入做互相关运算时，如果横向相邻元素相同，输出为0，否则输出为非0

In [18]:
K = torch.tensor([[1,-1]])

#### 下面将输入X和我们设计的卷积核K做互相关运算。可以看出，我们将从白到 黑的边缘和从黑到白的边缘分别检测成了1和 -1,其余 部分的输出全是0

In [19]:
Y = corr2d(X,K)
print(Y)

tensor([[ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.]])


#### 由此，我们可以看出，卷积层可以通过重复使用卷积核有效的表征局部空间

### 通过数据学习核数组
#### 最后我们看一个例子，他使用物体边缘检测中的输入数据X和输出数据Y来学习我们构造的核数组K,我们首先构造一个卷积层，其卷积核将被初始化成随机数组，在接下来的每一次迭代中，我们使用平方误差来 比较Y和卷积层的输出。然后计算梯度来更新权重

In [29]:
#构造一个核数组形状是（1，2）的二维卷积层
conv2d = Conv2D(kernel_size = (1,2))

step = 20
lr = 0.01
for i in range(step):
    Y_hat = conv2d(X)
    l = ((Y_hat -Y)**2).sum()
    l.backward()
    
    #梯度下降
    
    conv2d.weight.data -= lr*conv2d.weight.grad
    conv2d.bias.data -= lr*conv2d.bias.grad
    
    #梯度清零
    conv2d.weight.grad.fill_(0)
    conv2d.bias.grad.fill_(0)
    if(i+1) % 5 == 0:
        print('step %d,loss %.3f' %(i+1,l.item()))

step 5,loss 4.966
step 10,loss 1.006
step 15,loss 0.238
step 20,loss 0.062


In [30]:
print('weight:',conv2d.weight.data)
print('bias:',conv2d.bias.data)

weight: tensor([[ 0.9465, -0.9306]])
bias: tensor([-0.0089])


#### 可以看到，学到的卷积核的权重参数与我们之前定义的核数组K较接近。而偏置参数接近0

### 互相关运算和卷积运算
#### 实际上，卷积运算与互相关运算类似。为了得到卷积运算的输出，我们只需要将核数组左右翻转并上下翻转，再与输入数组做互相关运算。可见，卷积运算和互相关运算虽然类似，但如果他们使用相同的核数组，对于同一个输入，输出往往并不相同

#### 那么，你也许会好奇卷积层为何能使用互相关运算代替卷积运算，其实，在深度学习中核数组都是学出来的：卷积层无论使用互相关运算或卷积运算都不影响模型预测时的输出。为了解释这一点，假设卷积层使用互相关运算学出图中的核数组。假设其他条件不变，使用卷积运算学出的核数组 即图中的核数组上下，左右翻转，也就是说，图中的输入与学出的已翻转的核数组再做卷积运算时，依然得到图中的输出。为了与大多数深度学习文献一致，如无特别说明，本书中提到的卷积运算均值互相关